In [ ]:
import tensorflow as tf
import numpy as np

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False

In [ ]:
feature_extractor = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D()
])

In [ ]:
def extract_features(dataset):
    features = []
    labels = []

    for images, y in dataset:
        images = tf.keras.applications.mobilenet_v2.preprocess_input(images)
        f = feature_extractor(images, training=False)
        features.append(f.numpy())
        labels.append(y.numpy())

    return np.vstack(features), np.concatenate(labels)

In [ ]:
#EXTRACT FEATURES 
X_train, y_train = extract_features(train_ds)
X_val, y_val     = extract_features(val_ds)
X_test, y_test   = extract_features(test_ds)

## AFTER THIS , WE SHOULD HAVE A NORMAL TUBULAR DATASET TO TRAIN ON ## 

In [ ]:
#SVM CLASSIFIER
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

svm = SVC(kernel="rbf", C=10, gamma="scale")
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
#XGBOOST CLASSIFIER
from xgboost import XGBClassifier

xgb = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8
)

xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)

print(classification_report(y_test, y_pred))
